### CAO Points Notebook

Include a Jupyter notebook called cao.ipynb that contains the following.

10% A clear and concise overview of how to load CAO points information from the
CAO website into a pandas data frame, pitched as your classmates.
***

20% A detailed comparison of CAO points in 2019, 2020, and 2021 using the functionality in pandas.
***

10% Appropriate plots and other visualisations to enhance your notebook for viewers.
***

#  Introduction - description

https://github.com/ianmcloughlin/cao-points/blob/main/cao-points-analysis.ipynb

### Packages required

In [1]:
# Importing required packages:
# HTTP Requests
import requests as rq

# Regular Expressions
import re

# Dates and times
import datetime as dt

# Data frames
import pandas as pd

# For Url downloads
import urllib.request as urlrq

import numpy as np

# Points loading:
***

## CAO Points for 2021 <br>
http://www.cao.ie/index.php?page=points&p=2021
***

In [2]:
# Getting data form CAO website
resp = rq.get("http://www2.cao.ie/points/l8.php")

# Confirming it is ok

resp

<Response [200]>

### Saving original dataset

In [3]:
# Getting current date and time
now = dt.datetime.now()

# Format as a string
nowstr = now.strftime("%Y%m%d_%H%M%S")

In [4]:
# Creating a file path
path2021 = 'data/cao2021_' + nowstr + '.html'

### Using decoding cp1252 that includes fada
***

In [5]:
# To include CAO website encoding screen 

In [6]:
# Checking original server encoding
original_encoding = resp.encoding

resp.encoding = 'cp1252'
original_encoding

'iso-8859-1'

In [7]:
# Save original html file
with open(path2021, 'w') as f:
    f.write(resp.text)

# Regular expression
***

In [8]:
# Compiling regular expression for line matching
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

In [9]:
# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


**Line count for CAO 2021 points is expected at 949.**

In [10]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [11]:
df2021

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN


## CAO Points for 2020 <br>
https://www.cao.ie/index.php?page=points&p=2020

http://www2.cao.ie/points/CAOPointsCharts2020.xlsx
***

In [12]:
# Setting ulr link or 2020 CAO points

url = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [13]:
# File path for to save original CAO 2020 file.
pathxlx = 'data/cao2020_' + nowstr + '.xlsx'

In [14]:
urlrq.urlretrieve(url, pathxlx)

('data/cao2020_20211222_224603.xlsx',
 <http.client.HTTPMessage at 0x1fb264ab100>)

In [15]:
url

'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [16]:
df2020 = pd.read_excel(url, skiprows=10)

In [17]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Spot checking one row
df2020.iloc[128]

CATEGORY (i.e.ISCED description)       Mathematics and statistics
COURSE TITLE                                Mathematical Sciences
COURSE CODE2                                                CK407
R1 POINTS                                                     566
R1 Random *                                                   NaN
R2 POINTS                                                     565
R2 Random*                                                      *
EOS                                                           565
EOS Random *                                                  NaN
EOS Mid-point                                                 601
LEVEL                                                           8
HEI                                 University College Cork (NUI)
Test/Interview #                                              NaN
avp                                                           NaN
v                                                             NaN
Column1   

In [19]:
# Spot checking last row
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [20]:
# File path for saving 2020 data
path2020 = 'data/cao2020' + nowstr + '.csv'


In [21]:
# Saving 2020 data to csv
df2020.to_csv(path2020)


## CAO Points for 2019 <br>
https://www.cao.ie/index.php?page=points&p=2019
http://www2.cao.ie/points/lvl8_19.pdf
***

Importing Camelot package for table reading  and extracting CAO 2019 points as tables from PDF, ########## [Camelot read all PDF](https://stackoverflow.com/questions/56777241/camelot-is-reading-only-the-first-page-of-the-pdf)

## Importing Camelot and extracting 2019 points

In [22]:
import camelot

df = camelot.read_pdf('http://www2.cao.ie/points/lvl8_19.pdf', pages='all')

df

<TableList n=18>

In [23]:
# A terrible list, need a loop or something better
alltables = [df[0].df, df[1].df, df[2].df, df[3].df, df[4].df, df[5].df, df[6].df, df[7].df, df[8].df, df[9].df, df[10].df, df[11].df, df[12].df, df[13].df, df[14].df, df[15].df, df[16].df, df[17].df]
# Merging tables together
df2019 = pd.concat(alltables)

# alltables = pd.concat([tables[0].df, tables[1].df])
df2019

,0,1,2,3
0,Course Code INSTITUTION and COURSE,,EOS,Mid
1,,Athlone Institute of Technology,,
2,AL801,Software Design with Virtual Reality and Gaming,304,328
3,AL802,Software Design with Cloud Computing,301,306
4,AL803,Software Design with Mobile Apps and Connected...,309,337
...,...,...,...,...
37,WD200,Arts (options),221,296
38,WD210,Software Systems Development,271,329
39,WD211,Creative Computing,275,322
40,WD212,Recreation and Sport Management,274,311


In [24]:
# File path for saving 2020 data
path2019 = 'data/cao2019' + nowstr + '.csv'


In [25]:
# Saving 2019 data to csv
df2019.to_csv(path2019)

# Joining data
***

In [26]:
# Selecting requied columns from 2021 dataframe
courses2021 = df2021[['code', 'title']]
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


CSV data needs to be reviewed to confirm all data points. 2020 point data has level 6, 7 and 8 together in as one data file that needs to be filtered out before merging with 2021 and 2019 data to avoid misrepresentation. Alternatively, level 6 and 7 courses can be be removed after all data is merged by removing all Nan values, as only level 8 courses will get matched. Yet only courses constantly across all years will be left removing data for for courses that were available just some of the year.

In [27]:
# Selecting only level 8 courses from 2020 data
lvl8df2020 = df2020[df2020['LEVEL'] == 8]
lvl8df2020


,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Arts,Arts (options),WD200,AQA,NaN,AQA,NaN,AQA,NaN,336,...,avp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Expected 2020 level 8 courses are 1027 as confirmed to filtered excel file.**

In [28]:
# Selecting requied columns from 2020 dataframe
courses2020 = lvl8df2020[['COURSE CODE2','COURSE TITLE']]
# Renaming columns to have same names across all years
courses2020.columns = ['code', 'title']
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1455,WD200,Arts (options)
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


CAO 2019 points data has rows with only institution name, this will be removed by assigning NAN value to emty fields and removing all rows containig NAN under course code.

In [32]:
# Selecting requied columns from 2019 dataframe 
courses2019 = df2019[[0,1]]
# Renaming columns to have same names across all years
courses2019.columns = ['code', 'title']
courses2019

,code,title
0,Course Code INSTITUTION and COURSE,
1,,Athlone Institute of Technology
2,AL801,Software Design with Virtual Reality and Gaming
3,AL802,Software Design with Cloud Computing
4,AL803,Software Design with Mobile Apps and Connected...
...,...,...
37,WD200,Arts (options)
38,WD210,Software Systems Development
39,WD211,Creative Computing
40,WD212,Recreation and Sport Management


# Analysis and CAO points comaparison

# Visualisations 

# Conclusion

### References

In [ ]:
# Camelot https://www.youtube.com/watch?v=LoiHI-IB3lY

https://camelot-py.readthedocs.io/en/master/index.html

https://dsevero.com/posts/pandas/

https://github.com/ianmcloughlin/cao-points/blob/main/cao-points-analysis.ipynb

https://nbviewer.org/github/ianmcloughlin/jupyter-teaching-notebooks/blob/main/regex.ipynb